<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the repository and cd into it

In [7]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data
!git pull

Cloning into 'evaluating-llm-generated-nlu-data'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 195 (delta 109), reused 131 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (195/195), 157.47 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/evaluating-llm-generated-nlu-data/evaluating-llm-generated-nlu-data
Already up to date.


Import the necessary classes and methods

In [8]:
from bin.data_generation.generate_data import DataGenerationModel
from bin.data_generation.construct_prompt import load_prompt
from bin.utils.types import DataSet
from transformers import AutoModelForCausalLM, AutoTokenizer

Initialize Phi-3.5 Model

In [9]:
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load the model

In [10]:
phi = DataGenerationModel(model=phi_model, tokenizer=phi_tokenizer)

Generate data with basic prompt

In [12]:
prompt_id = "one_shot_simple"
intents = ["ac_on", "ac_off", "light_on", "light_off", "music_on", "music_off"]
dataset = phi.build_dataset_from_intents(prompt_id, intents)
dataset.to_csv(f"{prompt_id}_data.csv", columns=["query", "intent"])
dataset.to_data_frame()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


TypeError: list indices must be integers or slices, not str